In [124]:
using Optim
using LinearAlgebra

function p_count(constraints)
    # Only have inequality constraints
    return sum(if constraints[i] == false 1 else 0 end for i in 1:length(constraints))
end

# Penalty method
function penalty_method(f, p, x, k_max; ρ=1, γ=2)

    steps = []
    evaluations = 0
    trace = []

    # convergence measure
    previous = [1e+10, 1e+10, 1e+10]
    
    for k in 1 : k_max
        push!(steps, x)
        x, evals, t = minimize(x -> f(x) + ρ*p(x), x)
        
        for i in 1:length(t)
            push!(trace, t[i])
        end
            
        evaluations += evals
        ρ *= γ

        measure = sum([(x[i] - previous[i])^2 for i in 1:3])
        if measure < 1e-6
            push!(steps, x)
            return (x, steps, evaluations, trace)
        end
        
        if p(x) == 0
            push!(steps, x)
            return (x, steps, evaluations, trace)
        end

        previous = x
    end
    return (x, steps, evaluations, trace)
end

function minimize(objective, a)
    res = optimize(objective, a, store_trace=true, extended_trace=true, )
    return (Optim.minimizer(res), Optim.iterations(res), Optim.centroid_trace(res))
end

# Singular asteroid constraint, returns T/F
function single_asteroid(rocket_pt, asteroid_pt, range)
    return sum([(rocket_pt[i] - asteroid_pt[i])^2 for i in 1:3]) >= range
end

# Data generation
# Function to generate random point within the given range
max_range = 1000.0
max_asteroids = 500
max_data = 100
max_size = 50000.0

function random_point()
    return [rand(-max_range:max_range), rand(-max_range:max_range), rand(-max_range:max_range)]
end

# Function to generate sequences
function generate_sequences()
    sequences = []
    for _ in 1:max_data
        start_pt = random_point()
        end_pt = random_point()
        asteroids = [random_point() for _ in 1:max_asteroids]
        sizes = [rand(100.0:max_size) for _ in 1:max_asteroids]
        sequence = Dict(
            "start_pt" => start_pt,
            "end_pt" => end_pt,
            "asteroids" => asteroids,
            "sizes" => sizes
        )
        push!(sequences, sequence)
    end
    return sequences
end

# Generate sequences
sequences = generate_sequences()

# Accessing a sample sequence to view the generated data
sample_sequence = sequences[1]
println("Sample Start Point: ", sample_sequence["start_pt"])
println("Sample End Point: ", sample_sequence["end_pt"])
# println("Sample Asteroids: ", sample_sequence["asteroids"])
# println("Sample Sizes: ", sample_sequence["sizes"])

Sample Start Point: [-141.0, 698.0, 714.0]
Sample End Point: [545.0, 236.0, 328.0]


In [125]:
# Here we go... Optimize dist function with the constraints
final_pts = []
true_error = []
evaluations = []
all_traces = []

for j in 1:length(sequences)

    start_pt = sequences[j]["start_pt"]
    end_pt = sequences[j]["end_pt"]
    A = sequences[j]["asteroids"]
    d = sequences[j]["sizes"]

    # Calculates difference between point x and this end point
    function f(x)
        sum([(x[i] - end_pt[i])^2 for i in 1:3])
    end

    # Calculates number of penalties applied for point x and all asteroids
    function p(x)
        return sum([if single_asteroid(x, A[i], d[i]) 0 else 1 end for i in 1:length(A)])
    end

    result, _, evals, trace = penalty_method(f, p, start_pt, 1000)
    
    error = sum([(result[i] - end_pt[i])^2 for i in 1:3])
    push!(final_pts, result)
    push!(all_traces, trace)
    push!(true_error, error)
    push!(evaluations, evals)

end

"""
println("Example output")
for i in 1:6

    println("START PT: ", sequences[i]["start_pt"])
    println("TRUE END PT: ", sequences[i]["end_pt"])
    println("CALCULATED END PT: ", final_pts[i])
    println("ERROR: ", true_error[i])
    println("F EVALS: ", evaluations[i])
    println("Trace -> ", all_traces[i])
    println("Asteroids -> ", sequences[i]["asteroids"])
    println()

end
"""

ratio_correct = sum([if true_error[i] > 1e-5 0 else 1 end for i in 1:length(true_error)]) / length(true_error)
println("Times complete path found: ", ratio_correct)

Times complete path found: 0.95


In [128]:
open("output.txt", "w") do io
    redirect_stdout(io) do
        for i in 1:max_data
            println(sequences[i]["start_pt"])
            println(sequences[i]["end_pt"])
            println(final_pts[i])
            println(true_error[i])
            println(evaluations[i])
            println(all_traces[i])
            println(sequences[i]["asteroids"])
        end
    end
end